In [719]:
import numpy as np

In [720]:
import math

def sort_points_clockwise(points):
    # Find the point with the smallest y-coordinate (ties broken by smallest x-coordinate)
    ref_point = min(points, key=lambda p: (p[1], p[0]))

    # Sort the points by polar angle with respect to the reference point
    sorted_points = sorted(points, key=lambda p: (math.atan2(p[1]-ref_point[1], p[0]-ref_point[0]), math.dist(ref_point, p)))

    return sorted_points

In [721]:
def remove_duplicates(points):
    point_set = set([tuple(x) for x in points])
    points = np.array(list(point_set))
    points = sort_points_clockwise(points)

    return points


In [722]:
def check_line(points):
    m = (points[-1][-1] - points[0][-1])/(points[-1][0] - points[0][0])
    b = points[-1][-1] - (m*points[-1][0])

    if points[1][1] == m*points[1][0] + b:
        return [points[0], points[-1]]
    else:
        return points
    

In [723]:
def remove_points_between(merged_hull, left_hull, right_hull):
    if len(left_hull.shape) == 1:
        left_hull = np.array([left_hull])
    if len(right_hull.shape) == 1:
        right_hull = np.array([right_hull])
    new_hull = [left_hull[0]]

    # Check each point on the merged hull in clockwise order
    for i in range(len(merged_hull)):
        # Check if the line segment between this point and the next point on the merged hull
        # intersects with the line segment between the first point on the left hull and
        # the first point on the right hull
        if is_left_turn(left_hull[0], right_hull[0], merged_hull[i]):
            # This point is on the boundary of the merged hull, so add it to the new list
            new_hull.append(merged_hull[i])

    # Add the last point on the right hull to the new list
    new_hull.append(right_hull[0])
    return new_hull

In [724]:
def next_point(chain, point, reverse=False):

    index = np.where(np.all(chain == point, axis=1))[0]
    if not reverse:
        index = (index + 1) % len(chain)
    else:
        index = (index - 1) % len(chain)
    index = index[0]
    return chain[index]

In [725]:
def is_left_turn(p1, p2, p3):
    return ((p2[0] - p1[0]) * (p3[1] - p1[1]) -
            (p2[1] - p1[1]) * (p3[0] - p1[0])) > 0

In [726]:
def lower_tangent(left, right):
    l_left = min(left, key=lambda p: p[1])
    l_right = min(right, key=lambda p: p[1])
    while True:
        if (l_left==min(left, key=lambda p: p[1])).all() is True:
            l = next_point(left, l_left, reverse=True)
            if is_left_turn(l_left, l_right, l):
                l_left = l
                continue
        if (l_right==min(right, key=lambda p: p[1])).all():
            r = next_point(right, l_right)
            if is_left_turn(r, l_right, l_left):
                l_right = r
                continue
        break
    return l_left, l_right

In [727]:
def merge_hulls(left, right):
    u_left, u_right = upper_tangent(left, right)
    l_left, l_right = lower_tangent(left, right)
    upper_chain = remove_points_between(left, u_left, u_right)
    lower_chain = remove_points_between(right, l_right, l_left)
    merged_hull = np.array(upper_chain + lower_chain)
    return merged_hull

def upper_tangent(left, right):
    u_left = max(left, key=lambda p: p[1])
    u_right = max(right, key=lambda p: p[1])
    while True:
        if (u_left==max(left, key=lambda p: p[1])).all() is True:
            l = next_point(left, u_left)
            if is_left_turn(u_right, u_left, l):
                u_left = l
                continue
        if (u_right==max(right, key=lambda p: p[1])).all() is True:
            r = next_point(right, u_right, reverse=True)
            if is_left_turn(u_left, u_right, r):
                u_right = r
                continue
        break
    return u_left, u_right



In [728]:
def convex_hull_2d(points: np.array):
    if len(points) < 3:
        return points
    else:
        midpoint = int(len(points)/2)
        left = convex_hull_2d(points[:midpoint])
        right = convex_hull_2d(points[midpoint:])
        return merge_hulls(left,right)

In [729]:
convex_hull_2d(np.array([(1,2),(9,10),(19,10),(10,8),(7,5),(8,19),(4,6)]))

[array([1, 2]),
 array([19, 10]),
 array([7, 5]),
 array([10,  8]),
 array([ 8, 19])]

In [5]:
import math

def convex_hull_2d(points):
    if len(points) < 3:
        return points
    first_point = min(points)
    sorted_points = sorted(points, key=lambda p: math.atan2(p[1]-first_point[1], p[0]-first_point[0]))
    mid = len(sorted_points) // 2
    left_points = sorted_points[:mid]
    right_points = sorted_points[mid:]
    left_hull = convex_hull_2d(left_points)
    right_hull = convex_hull_2d(right_points)
    return merge_hulls(left_hull, right_hull)

In [6]:
def merge_hulls(left_hull, right_hull):
    # Find the leftmost and rightmost points of the hulls
    leftmost_point = min(left_hull)
    rightmost_point = max(right_hull)

    # Move leftmost_point and rightmost_point toward each other
    # until they are both on the hull
    while True:
        # Find the point after leftmost_point in left_hull
        if left_hull.index(leftmost_point) == len(left_hull) - 1:
            a = left_hull[0]
        else:
            a = left_hull[left_hull.index(leftmost_point) + 1]
        # Find the point before rightmost_point in right_hull
        if right_hull.index(rightmost_point) == 0:
            b = right_hull[-1]
        else:
            b = right_hull[right_hull.index(rightmost_point) - 1]
        # If a is above the line between leftmost_point and b,
        # move leftmost_point to a
        if is_above_line(a, leftmost_point, b):
            leftmost_point = a
        else:
            # Otherwise, break out of the loop
            break
    while True:
        # Find the point after rightmost_point in right_hull
        if right_hull.index(rightmost_point) == len(right_hull) - 1:
            a = right_hull[0]
        else:
            a = right_hull[right_hull.index(rightmost_point) + 1]
        # Find the point before leftmost_point in left_hull
        if left_hull.index(leftmost_point) == 0:
            b = left_hull[-1]
        else:
            b = left_hull[left_hull.index(leftmost_point) - 1]
        # If a is above the line between rightmost_point and b,
        # move rightmost_point to a
        if is_above_line(a, rightmost_point, b):
            rightmost_point = a
        else:
            # Otherwise, break out of the loop
            break
    # Combine the hulls
    result = []
    for point in left_hull:
        if point == leftmost_point:
            break
        result.append(point)
    result.append(leftmost_point)
    for point in right_hull:
        if point == rightmost_point:
            break
        result.append(point)
    result.append(rightmost_point)
    return result

In [7]:
def is_above_line(test_point, line_start, line_end):
    # Calculate the slope and y-intercept of the line
    slope = (line_end[1] - line_start[1]) / (line_end[0] - line_start[0])
    y_intercept = line_start[1] - slope * line_start[0]
    # Calculate the y-coordinate of the line at the x-coordinate of test_point
    y_on_line = slope * test_point[0] + y_intercept
    # If the y-coordinate of test_point is above the line, return True
    if test_point[1] > y_on_line:
        return True
    # Otherwise, return False
    else:
        return False


In [8]:
points = [(0, 0), (1, 1), (2, 2), (2, 0), (3, 3), (4, 0), (5, 1), (5, 3)]

convex_hull_2d(points)

[(0, 0), (1, 1), (2, 2), (3, 3)]

In [3]:
# A divide and conquer program to find convex
# hull of an given set of points.
 
# stores the centre of polygon (It is made
# global because it is used in find_corner function)
reference_point = [0, 0]

 
# find_corner function for sorting
def find_corner(p1, q1):
    p = [p1[0] - reference_point[0], p1[1] - reference_point[1]]
    q = [q1[0] - reference_point[0], q1[1] - reference_point[1]]
    cross_product = p[0] * q[1] - p[1] * q[0]
    if cross_product > 0:
        return -1
    elif cross_product < 0:
        return 1
    else:
        return 0

 
# Finds upper tangent of two polygons 'lower_hull' and 'upper_hull'
# represented as two vectors.
def merge_hull(lower_hull, upper_hull):
    p = len(lower_hull)
    q = len(upper_hull)

    leftmost1 = lower_hull.index(max(lower_hull, key=lambda x: x[0]))
    leftmost2 = upper_hull.index(min(upper_hull, key=lambda x: x[0]))
 
    check_tangent = lambda p1, upper_hull, c: (upper_hull[1]-p1[1]) * (c[0]-upper_hull[0]) - (c[1]-upper_hull[1]) * (upper_hull[0]-p1[0])

    # find upper tangent
    i = next((i for i in range(p) if check_tangent(upper_hull[leftmost2], lower_hull[leftmost1], lower_hull[(leftmost1+1) % p]) >= 0), leftmost1)
    j = next((j for j in range(q) if check_tangent(lower_hull[i], upper_hull[leftmost2], upper_hull[(q+leftmost2-1) % q]) <= 0), leftmost2)

    uppera, upperb = i, j

    # find lower tangent
    j = next((j for j in range(p) if check_tangent(lower_hull[leftmost1], upper_hull[leftmost2], upper_hull[(leftmost2+1) % q]) >= 0), leftmost2)
    i = next((i for i in range(q) if check_tangent(upper_hull[j], lower_hull[leftmost1], lower_hull[(p+leftmost1-1) % p]) <= 0), leftmost1)

    lowera, lowerb = i, j

    final = []
    # if lowera < uppera:
    #     final = lower_hull[uppera:] + lower_hull[:lowera+1]
    # else:
    #     final = lower_hull[uppera:lowera+1]
    
    # ind = lowerb
    # final.append(upper_hull[lowerb])
    # while ind != upperb:
    #     ind = (ind+1) % q
    #     final.append(upper_hull[ind])

    # merge the two hulls *CHECK*
    if lowera < uppera:
        final = lower_hull[uppera:lowera+1] + upper_hull[lowerb:upperb+1]
    else:
        final = upper_hull[lowerb:upperb+1] + lower_hull[uppera:lowera+1]
    return final
 

def bruteHull(lower_hull):
    global reference_point
    s = set()
    for i in range(len(lower_hull)):
        for j in range(i+1, len(lower_hull)):
            x1, x2 = lower_hull[i][0], lower_hull[j][0]
            y1, y2 = lower_hull[i][1], lower_hull[j][1]
            a1, b1, c1 = y1-y2, x2-x1, x1*y2-y1*x2
            pos, neg = 0, 0
            for k in range(len(lower_hull)):
                if a1*lower_hull[k][0]+b1*lower_hull[k][1]+c1 <= 0:
                    neg += 1
                if a1*lower_hull[k][0]+b1*lower_hull[k][1]+c1 >= 0:
                    pos += 1
            if pos == len(lower_hull) or neg == len(lower_hull):
                s.add(tuple(lower_hull[i]))
                s.add(tuple(lower_hull[j]))
 
    final = []
    for x in s:
        final.append(list(x))
 
    # Sorting the points in the anti-clockwise order
    reference_point = [0, 0]
    n = len(final)
    for i in range(n):
        reference_point[0] += final[i][0]
        reference_point[1] += final[i][1]
        final[i][0] *= n
        final[i][1] *= n
    from functools import cmp_to_key
    final = sorted(final, key=cmp_to_key(find_corner))
    for i in range(n):
        final[i] = [final[i][0]/n, final[i][1]/n]
    return final
 
# Returns the convex hull for the given set of points
def convex_hull_2d(points):
    if len(points) < 5:
        return bruteHull(points)
    else:
        midpoint = int(len(points)/2)
        left_hull = convex_hull_2d(points[:midpoint])
        right_hull = convex_hull_2d(points[midpoint:])

    return merge_hull(left_hull, right_hull)
 
# Driver Code
if __name__ == '__main__':
    c = [(1,2),(9,10),(19,10),(8,19),(4,6),(7,5)]
 
    n = len(c)
    # sorting the set of points according
    # to the x-coordinate
    c.sort()
    ans = convex_hull_2d(c)
 
    print('Convex Hull:')
    for x in ans:
        print(int(x[0]), int(x[1]))

Convex Hull:
1 2
19 10
8 19
